In [1]:

#matファイル名
matname = 'results_shikata.mat'
#保存ファイル名
filename = '50'


In [2]:
from scipy.io import loadmat # type: ignore
import pandas as pd
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# .matファイルからデータを読み込む (loadmatをインポート・使用)

data = loadmat(f'{matname}')
#CSVファイルをpandasのread_csvで読み込む
df_feederNodeName = pd.read_csv('Hnode.csv')
#df_pvposition = pd.read_csv('PVposition.csv')

# データのキーを表示(表の一番上の行に履いている系列のようなもの？)
print(data.keys())
# データフレームの表示
#print(df_merged)

# 特定の変数（行列データ）を取得 from results.mat
MVload_data = data["P_MVload"]
MVpv_data = data["P_MVpv"]
CSload_data = data["P_cs"]
MVcurrent_data = data["Ir_MV"]

loadName_data = data['Name_MVload']
pvName_data = data["Name_MVpv"]
csName_data = data["Name_CS"]

# 行列データをPandasのデータフレームに変換)
df_MVload = pd.DataFrame(MVload_data)
df_MVpv = pd.DataFrame(MVpv_data)
df_CSload = pd.DataFrame(CSload_data)
df_MVcurrent = pd.DataFrame(MVcurrent_data)

#.Tによって転置したものを変換
df_loadName = pd.DataFrame(loadName_data).T
df_pvName = pd.DataFrame(pvName_data).T
df_csName = pd.DataFrame(csName_data).T

dict_keys(['__header__', '__version__', '__globals__', 'Name_MVnode', 'Name_MVline', 'Name_MVload', 'Name_MVpv', 'Name_CS', 'Name_oltc', 'Name_svr', 'Name_pXfmr', 'Name_Mon', 'Name_rtMon', 'Vrs_MV', 'Ir_MV', 'I_MV_3ph', 'I_MV_ctrl', 'Ploss_MVline', 'Qloss_MVline', 'P_MVload', 'P_MVload_test', 'Q_MVload', 'P_MVpv', 'Q_MVpv', 'P_cs', 'P_chrg', 'Loss_System', 'loss_per_day', 'OLTC_Params', 'Vrs_oltc_pri', 'Vrs_oltc_sec', 'Ir_oltc_pri', 'Irang_oltc_pri', 'Ir_oltc_sec', 'Irang_oltc_sec', 'OLTC_tap', 'OLTC_tapPos', 'Vrs_svr_sec', 'Vst_svr_sec', 'Vtr_svr_sec', 'Ir_svr_sec', 'Irang_svr_sec', 'Is_svr_sec', 'It_svr_sec', 'SVR_tap', 'SVR_tapPos', 'P_pXfmrs', 'Q_pXfmrs', 'Ploss_pXfmrs', 'Qloss_pXfmrs', 'PF_MVpv', 'loss_18_24'])


In [3]:
import pandas as pd

# リスト内の文字列を置換する関数を定義
def replace_strings_in_list(entry, old_str, new_str):
    return [x.replace(old_str, new_str) for x in entry]

# 文字列の置換を一括して行う関数を定義
def replace_strings(df, old_str, new_str):
    df[0] = df[0].apply(lambda entry: replace_strings_in_list(entry, old_str, new_str))
    return df.T

# 各データフレームに対して文字列置換を実行
df_loadName = replace_strings(df_loadName, "hl", "hn")
df_pvName = replace_strings(df_pvName, "hg", "hn")
df_csName = replace_strings(df_csName, "cs", "hn")

# データを結合(縦方向かつ、縦方向のインデックスを無視して0からインデックスを振りなおす)
df_merged_load = pd.concat([df_loadName, df_MVload], ignore_index=True)
df_merged_pv = pd.concat([df_pvName, df_MVpv], ignore_index=True)
df_merged_cs = pd.concat([df_csName, df_CSload], ignore_index=True)
df_merged_current = pd.concat([df_loadName, df_MVcurrent], ignore_index=True)

# 列名を最初の行に設定
def set_columns_and_drop_first_row(df):
    df.columns = df.iloc[0].astype(str)
    return df.drop(df.index[0])

df_merged_load = set_columns_and_drop_first_row(df_merged_load)
df_merged_pv = set_columns_and_drop_first_row(df_merged_pv)
df_merged_cs = set_columns_and_drop_first_row(df_merged_cs)
df_merged_current = set_columns_and_drop_first_row(df_merged_current)

print(df_merged_load)
print(df_merged_pv)
print(df_merged_cs)
print(df_merged_current)

# 列名を新しい行として保存し、重複を削除
new_row = pd.concat([
    pd.DataFrame([df_merged_load.columns], columns=df_merged_load.columns),
    pd.DataFrame([df_merged_pv.columns], columns=df_merged_pv.columns),
    pd.DataFrame([df_merged_cs.columns], columns=df_merged_cs.columns),
    pd.DataFrame([df_merged_current.columns], columns=df_merged_current.columns)
], axis=1).loc[:, ~pd.concat([
    pd.DataFrame([df_merged_load.columns], columns=df_merged_load.columns),
    pd.DataFrame([df_merged_pv.columns], columns=df_merged_pv.columns),
    pd.DataFrame([df_merged_cs.columns], columns=df_merged_cs.columns),
    pd.DataFrame([df_merged_current.columns], columns=df_merged_current.columns)
], axis=1).columns.duplicated()]

print(new_row)
# データ型を適切に変更
df_merged_load = df_merged_load.astype(float)
df_merged_pv = df_merged_pv.astype(float)
df_merged_cs = df_merged_cs.astype(float)
df_merged_current = df_merged_current.astype(float)

# 3つのデータフレームを結合
combined_df = pd.concat([df_merged_load, df_merged_pv, df_merged_cs], axis=1)

# 列名に基づいて集約し、合算を行う
df_merged = combined_df.groupby(by=combined_df.columns, axis=1).sum()

# 新しい行を追加
df_merged = pd.concat([new_row, df_merged], ignore_index=True)

# カラム名を0, 1, 2, ...の数字に更新
df_merged.columns = range(len(df_merged.columns))

# 最初の行の文字列を小文字に変換
df_merged.iloc[0] = df_merged.iloc[0].apply(lambda x: str(x).lower())

# 結果を表示
print(df_merged)
print(df_merged_current)

# # CSVファイルに保存
#df_merged.to_csv("df_mergedload.csv", index=False)

0    ['hn825000010'] ['hn825000006'] ['hn825000004e07n2'] ['hn825000004e15']  \
1          60.663351        6.611611            47.309298          12.923553   
2          60.624258        6.611182            47.310036          12.915227   
3          60.584885        6.610724            47.310551           12.90684   
4          60.545511        6.610265            47.311069          12.898453   
5          60.506144        6.609806            47.311586          12.890066   
...              ...             ...                  ...                ...   
9716       50.847685        6.990704            53.908413          10.832385   
9717       50.866013        6.991477            53.919062          10.836288   
9718        50.88434        6.992251            53.929705          10.840192   
9719       50.902668        6.993025            53.940348          10.844095   
9720       50.920996        6.993799            53.950991          10.847998   

0    ['hn828160014s05e6s2'] ['hn8281600

C:\Users\Shin\AppData\Local\Temp\ipykernel_11804\70078338.py:62: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_merged = combined_df.groupby(by=combined_df.columns, axis=1).sum()


ノード名処理

In [4]:
def replace_space(value):
    if isinstance(value, str):
        value=value.replace(" ", "")#文字列entryに対して処理を行う
    return value


def replace_Parentheses(value):
    for item in ["[","]","'"]:
        # print(item)
        if isinstance(value, str):
            
            value=value.replace(item, "")#文字列entryに対して処理を行う
           
    return value


df_merged_cs.columns = df_merged_cs.columns.to_series().apply(replace_Parentheses)
df_merged_pv.columns = df_merged_pv.columns.to_series().apply(replace_Parentheses)
df_merged_load.columns = df_merged_load.columns.to_series().apply(replace_Parentheses)
df_merged= df_merged.T
df_merged[0] = df_merged[0].apply(replace_Parentheses)
df_merged= df_merged.T
print(df_merged)


              0            1                 2               3    \
0     hn825000010  hn825000006  hn825000004e07n2  hn825000004e15   
1       60.663351     6.611611         47.309298       12.923553   
2       60.624258     6.611182         47.310036       12.915227   
3       60.584885     6.610724         47.310551        12.90684   
4       60.545511     6.610265         47.311069       12.898453   
...           ...          ...               ...             ...   
9716    50.847685     6.990704         53.908413       10.832385   
9717    50.866013     6.991477         53.919062       10.836288   
9718     50.88434     6.992251         53.929705       10.840192   
9719    50.902668     6.993025         53.940348       10.844095   
9720    50.920996     6.993799         53.950991       10.847998   

                     4                   5            6               7    \
0     hn828160014s05e6s2  hn828160014s05e4s6  hn825000007  hn828160014s05   
1              25.962768     

In [5]:

# CSVファイルに保存 
df_merged_cs.to_csv(f"df_cs_{filename}.csv", index=False)
df_merged_load.to_csv(f"df_homeload_{filename}.csv", index=False)
df_merged_pv.to_csv(f"df_pv_{filename}.csv", index=False)
df_merged.to_csv(f"df_mergedload_{filename}.csv", index=False)

In [6]:
df_location = pd.read_csv('Hnode.csv')
def replace_Parentheses(value):
    for item in ["[","]","'"]:
        # print(item)
        if isinstance(value, str):
            
            value=value.replace(item, "")#文字列entryに対して処理を行う
           
    return value


def replace_space(value):
    if isinstance(value, str):
        value=value.replace(" ", "")#文字列entryに対して処理を行う
    return value

#半角スペースの削除、アルファベットの小文字化を行っておく
df_location['Node'] = df_location['Node'].str.lower()

df_location['Node']=df_location['Node'].apply(replace_space)



#df_locationのインデックスをノード名にしておく(後で照らし合わせに使うため)

df_location =  df_location.set_index('Node', drop=True)
print(df_location)

df_location.to_csv("df_location_processed.csv", index=True)


                   Latitude   Longitude Section  Feeder
Node                                                   
ss                34.803331  134.817406     NaN     NaN
hn827520054       34.825770  134.814790       7    25.0
hn827520087       34.829990  134.821230       8    25.0
hn827520088       34.830080  134.821580       8    25.0
hn828010001       34.824590  134.821800       8    25.0
...                     ...         ...     ...     ...
hn827230057e12    34.833350  134.839780       6    21.0
hn827230057e14    34.832630  134.839440       6    21.0
hn829992017k28k1  34.799890  134.829630       -    28.0
hn829992017k28k2  34.799240  134.829590       -    28.0
hn829992017k26    34.799930  134.829640       -    26.0

[3044 rows x 4 columns]
